In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.insert(0,'/home/bradhakrishnan/ECE276A_PR1/code/')

In [4]:
import load_data
import os
# load_data.read_data()
data_base_path= "/home/bradhakrishnan/ECE276A_PR1/data/"
 

# CALIBRATION Tasks 

1. Figure out how long the setup is static to measure the bias.
2. Once bias computed use the Value formula to find the values.
3. DONT FORGET TO CONVERT THE UNITS.

In [94]:
import jax
import jax.numpy as jnp

imud, vicd, camd=load_data.load_dataset(3)
vicd_data = jnp.array(vicd['rots'])
vicd_ts = jnp.array(vicd['ts'])


Data import took: 0.0023980140686035156 sec.



In [96]:
find_stationary_end(vicd_data, threshold=0.001, debug=False)
vicd_data[:,:, 0]

Array([[ 0.9963586 ,  0.08375555, -0.01595546],
       [-0.0837184 ,  0.996485  ,  0.00298392],
       [ 0.0161493 , -0.00163729,  0.9998683 ]], dtype=float32)

In [97]:
vicd_data[:,:, 1]

Array([[ 0.99635446,  0.08377246, -0.01612229],
       [-0.08373666,  0.99648374,  0.00288414],
       [ 0.01630722, -0.0015236 ,  0.9998659 ]], dtype=float32)

In [108]:
def if_rotation_identity(rotation, threshold=0.001, debug=False):
    diagonal = jnp.diag(rotation)
    if debug:
        print(jnp.abs(diagonal - 1.0))
    is_diagonal_close_to_one = jnp.all(jnp.abs(diagonal - 1.0) < threshold)
    if debug:
        print(is_diagonal_close_to_one)
    return is_diagonal_close_to_one

def find_stationary_end(vicd_data, threshold=0.001, debug=False):
    idx=0
    while(if_rotation_identity(vicd_data[:,:, idx], threshold, debug=debug)):
        idx+=1
    if debug:
        print(idx)
    return idx


def get_static_data(vicd_data, vicd_ts, imu_data, indentity_check_threshold=0.001, debug=False):
    stationary_end_idx= find_stationary_end(vicd_data, threshold=indentity_check_threshold, debug=debug)
    print("Number of Static entries:: ", stationary_end_idx)
    stationary_start_time= vicd_ts[:,0][0]  
    stationary_end_time= vicd_ts[:,stationary_end_idx][0]
    static_imu_data=imu_data[:,0:stationary_end_idx]
    if stationary_start_time== static_imu_data[0, 0] and stationary_end_time==static_imu_data[0,stationary_end_idx-1]:
        print(f"We might have to match the timestamps :: \nVidcon start:: {stationary_start_time} \nIMU start:: {static_imu_data[0, 0]}" )
        print(f"\n\nVidcon end:: {stationary_end_time} \nIMU end:: {static_imu_data[0, stationary_end_idx-1]}" )

    return static_imu_data

In [168]:
all_static_data= None
for ii in range(1,10):
    print(f"Dataset {ii} :: \n")
    imud, vicd, camd=load_data.load_dataset(ii)
    vicd_data = jnp.array(vicd['rots'])
    vicd_ts = jnp.array(vicd['ts'])
    static_imu_data= get_static_data(vicd_data, vicd_ts, imu_data, indentity_check_threshold=0.01, debug=False)
    if all_static_data is None:
        all_static_data=static_imu_data
    else:
        all_static_data= jnp.concatenate((all_static_data, static_imu_data), axis=1)

bias= jnp.mean(static_imu_data[1:, :], axis=1)
# we need to manually compute the bias for 
print(f"Bias ::",bias)


Dataset 1 :: 

Data import took: 0.013324260711669922 sec.

Number of Static entries::  1082
Dataset 2 :: 

Data import took: 0.0031211376190185547 sec.

Number of Static entries::  902
We might have to match the timestamps :: 
Vidcon start:: 1296637056.0 
IMU start:: 1296637056.0


Vidcon end:: 1296637184.0 
IMU end:: 1296637184.0
Dataset 3 :: 

Data import took: 0.0028536319732666016 sec.

Number of Static entries::  618
Dataset 4 :: 

Data import took: 0.0020623207092285156 sec.

Number of Static entries::  697
Dataset 5 :: 

Data import took: 0.002074003219604492 sec.

Number of Static entries::  503
Dataset 6 :: 

Data import took: 0.0019979476928710938 sec.

Number of Static entries::  430
Dataset 7 :: 

Data import took: 0.0022077560424804688 sec.

Number of Static entries::  495
Dataset 8 :: 

Data import took: 0.0021140575408935547 sec.

Number of Static entries::  748
Dataset 9 :: 

Data import took: 0.001714944839477539 sec.

Number of Static entries::  637
Bias :: [-511.   

In [181]:

## SENSITIVITY 
accl_sensitivity_all_axis=300 #mV/g
yaw_sensitivity = 3.33*(180/jnp.pi) #mV/ °/s
roll_sensitivity = 3.33*(180/jnp.pi)
pitch_sensitivity = 3.33*(180/jnp.pi)

## REFERENCE VOLTAGES
accl_vref= 3.3*1000
yaw_vref= 1.23*1000 #z axis
roll_vref=1.23*1000 # x axis
pitch_vref=1.23*1000 # y axis



In [197]:
# in the bias list the value we get z axis linear acceleration is basically the measurement for 9.8 m/s( i.e; g) acceleration
# so we can mathematically compute the bias using the avg(IMU measurement for 1g)-(1/scale_factor)
acc_z_bias= bias[2]- 1/((accl_vref/1023)/accl_sensitivity_all_axis)
bias= bias.at[2].set(acc_z_bias)
acc_x= (all_static_data[1,:]- bias[0]) * ((accl_vref/1023)/accl_sensitivity_all_axis)
acc_y= (all_static_data[2,:]- bias[1]) * ((accl_vref/1023)/accl_sensitivity_all_axis)
acc_z= (all_static_data[3,:]- bias[2]) * ((accl_vref/1023)/accl_sensitivity_all_axis)

w_x= (all_static_data[4,:]- bias[3]) * ((roll_vref/1023)/roll_sensitivity)
w_y= (all_static_data[5,:]- bias[4]) * ((pitch_vref/1023)/pitch_sensitivity)
w_z= (all_static_data[6,:]- bias[5]) * ((yaw_vref/1023)/yaw_sensitivity)

# ORIENTATION Tracking

MOTION model => qt+1 = f(qt, τtωt) := qt ◦ exp([0, τtωt/2])
OBSERVATION model => [0, at] = h(qt) := (qt^-1) ◦ [0, 0, 0, −g] ◦ qt]

1. The starting point is q0 = [1, 0, 0, 0],
2.  